# Оптимизация выбора лучшего жениха методом обучения с подкреплением (Reinforcement Learning)

Задача:
1. Невеста ищет себе жениха (существует единственное вакантное место).
2. Есть известное число претендентов — N.
3. Невеста общается с претендентами в случайном порядке, с каждым не более одного раза.
4. О каждом претенденте известно, лучше он или хуже любого из предыдущих.
5. Пообщавшись с претендентом, невеста сравнивает его с предыдущими и либо отказывает, либо принимает его предложение. Если предложение принято, они женятся и процесс останавливается. Если невеста отказывает жениху, то вернуться к нему позже она не сможет.
6. Невеста выигрывает, если она выберет самого лучшего претендента. Выбор даже второго по порядку сравнения — проигрыш.

Требуется найти решение, с наибольшей вероятностью приводящее к выбору самого лучшего претендента.

# Формулировка задачи:

Невеста должна выбрать лучшего жениха среди N претендентов, с каждым из которых она общается только один раз.

Она должна либо принять, либо отклонить жениха на месте.

Она выигрывает, если выберет самого лучшего претендента.

## Импорт необходимых библиотек

Для решения задачи будем использовать следующие библиотеки: gym для создания среды, stable_baselines3 для алгоритмов усиленного обучения, numpy для работы с массивами данных и json для сохранения результатов.

# Создание среды:

Определим среду для нашей задачи. Среда будет содержать список женихов с случайными рейтингами. Невеста будет принимать решение о каждом женихе по очереди, основываясь на стратегии, обученной с использованием методов обучения с подкреплением.

Мы создаем симуляционную среду BrideEnv с помощью библиотеки gym.

В этой среде претенденты появляются в случайном порядке с разными рейтингами.

Невеста может принять или отклонить каждого претендента.

Определение действий и наблюдений:

Действия: 0 - отклонить претендента, 1 - принять претендента.

Наблюдения: рейтинг текущего претендента.

## Метод reset

Метод `reset` подготавливает новую случайную последовательность женихов, устанавливает начальные значения переменных и возвращает рейтинг первого жениха для принятия решения.

## Метод step

Метод `step` реализует логику принятия решения агентом. Если агент принимает жениха, проверяется, является ли он лучшим. В случае отказа переходим к следующему жениху.

# Модель обучения:

Мы используем алгоритм PPO из библиотеки stable_baselines3 для обучения модели.

Модель обучается на множественных итерациях, чтобы оптимизировать стратегию выбора жениха.

# Обучение модели:

Модель повторяет процесс выбора жениха много раз, чтобы найти оптимальную стратегию.

В каждом эпизоде модель принимает решение для каждого претендента, и обучается на основе полученной награды (выбор лучшего претендента).

# Процесс выбора лучшего жениха
Процесс выбора лучшего жениха с использованием обученной модели. Агент принимает решение о выборе или отказе от каждого жениха и обновляет состояние среды.

# Оценка и проверка:

После обучения, мы проверяем, как модель принимает решения в новых сценариях.

Сохраняем данные об итерациях и шагах в JSON-файл для дальнейшего анализа.

In [4]:
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO
import json

class BrideEnv(gym.Env):
    def __init__(self, num_grooms=10):
        super(BrideEnv, self).__init__()
        self.num_grooms = num_grooms  # Количество женихов
        self.action_space = spaces.Discrete(2)  # Пространство действий: 0 - Отказать, 1 - Принять
        self.observation_space = spaces.Box(low=1, high=num_grooms, shape=(1,), dtype=np.int32)  # Пространство наблюдений
        self.reset()
    
    def reset(self):
        self.grooms = np.random.permutation(self.num_grooms) + 1  # Перемешиваем женихов и назначаем им рейтинги
        self.current_index = 0  # Начинаем с первого жениха
        self.done = False  # Устанавливаем, что процесс выбора не завершен
        self.max_groom = max(self.grooms)  # Находим жениха с максимальным рейтингом
        self.selected_groom = -1  # Изначально жених не выбран
        return np.array([self.grooms[self.current_index]])  # Возвращаем обсервацию
    
    def step(self, action):
        try:
            if action == 1:  # Если агент решил принять жениха
                self.selected_groom = self.grooms[self.current_index]  # Сохраняем выбранного жениха
                reward = 1 if self.selected_groom == self.max_groom else 0  # Награда за выбор лучшего жениха
                self.done = True  # Завершаем процесс выбора
                return np.array([self.selected_groom]), reward, self.done, {}  # Возвращаем выбранного жениха если процесс завершен
            else:  # Если агент решил отклонить жениха
                reward = 0  # Награда 0 за отклонение
                self.current_index += 1  # Переходим к следующему жениху
                if self.current_index < self.num_grooms:
                    return np.array([self.grooms[self.current_index]]), reward, False, {}  # Возвращаем новое состояние
                else:
                    self.done = True  # Завершаем процесс выбора
                    return np.array([self.selected_groom]), reward, self.done, {}  # Возвращаем пустое состояние, так как больше женихов нет
        except Exception as e:
            return None, 0, True, {"error": str(e)}  # Обрабатываем ошибки

    def render(self, mode='human'):
        pass

# Создаем среду
env = BrideEnv(num_grooms=10)

# Используем алгоритм PPO для обучения модели
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)  # Обучаем модель

# Функция для конвертации numpy-объектов в Python-объекты
def convert_numpy_to_python(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.integer):
        return int(data)
    elif isinstance(data, np.floating):
        return float(data)
    elif isinstance(data, np.bool_):
        return bool(data)
    elif isinstance(data, dict):
        return {k: convert_numpy_to_python(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_numpy_to_python(i) for i in data]
    else:
        return data

# Начинаем процесс выбора лучшего жениха
best_groom_found = False
iteration = 0
log_data = []

while not best_groom_found:
    iteration += 1
    obs = env.reset()  # Сбрасываем состояние среды
    done = False
    iteration_data = {
        "iteration": iteration,
        "steps": []
    }
    while not done:
        action, _states = model.predict(obs)  # Модель делает предсказание действия
        obs, rewards, done, info = env.step(action)  # Выполняем действие и получаем результат
        correct_choice = (env.selected_groom == env.max_groom) if action == 1 else False  # Проверяем правильность выбора
        step_data = {
            "observation": convert_numpy_to_python(obs.tolist()),  # Конвертируем обсервацию
            "action": int(action),  # Действие агента
            "reward": convert_numpy_to_python(rewards),  # Награда
            "done": convert_numpy_to_python(done),  # Завершено ли действие
            "info": convert_numpy_to_python(info),  # Дополнительная информация
            "current_index": convert_numpy_to_python(env.current_index),  # Текущий индекс жениха
            "grooms": convert_numpy_to_python(env.grooms.tolist()),  # Список женихов
            "max_groom": convert_numpy_to_python(env.max_groom),  # Рейтинг лучшего жениха
            "correct_choice": convert_numpy_to_python(correct_choice)  # Правильность выбора
        }
        iteration_data["steps"].append(step_data)
        if "error" in info:
            iteration_data["error"] = info["error"]
    log_data.append(iteration_data)
    if rewards == 1 and correct_choice:
        best_groom_found = True  # Найден лучший жених
        print(f"Обученная модель выбрала лучшего жениха с рейтингом {obs[0]} после {iteration} итераций")
    else:
        print(f"Итерация {iteration}: Модель ошиблась с выбором жениха и направлена на переобучение. Обсервация: {obs}, Рейтинг: {env.grooms}, Награда: {rewards}.")
        model.learn(total_timesteps=10000)  # Дополнительное обучение модели

# Сохраняем данные об итерациях в файл
try:
    with open("iteration_log.json", "w") as f:
        json.dump(convert_numpy_to_python(log_data), f, ensure_ascii=False, indent=4)
        print("Данные успешно сохранены в iteration_log.json")
except Exception as e:
    print(f"Ошибка при сохранении данных в файл: {e}")



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.96     |
|    ep_rew_mean     | 0.08     |
| time/              |          |
|    fps             | 2464     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.3          |
|    ep_rew_mean          | 0.13         |
| time/                   |              |
|    fps                  | 1231         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0110673765 |
|    clip_fraction        | 0.0756       |
|    clip_range           | 0.2          |
|    en